In [1]:
import pandas as pd
import numpy as np
import ampal
from dp_utils.pipeline_data import get_full_pipeline_df
from utils import useful_cols,useful_cols_non_csv,sharing_cols
import pandas

pandas.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:

def generate_deltaprot_designs_data():

    no_disulfide_df =pd.read_csv("/home/tadas/code/deltaproteinsBristol/selected_deltaprots/no_disulfide/no_disulfide_selected_deltaprots.csv")
    no_disulfide_whole_pipeline_df = get_full_pipeline_df("/home/tadas/code/deltaproteinsBristol/pipeline_data/05_evaluation")
    # merge no_disulfide_whole_pipeline_df into no_disulfide_df only where by model_sequence and sequence_name match. Also ignore duplicate columns
    no_disulfide_df = pd.merge(no_disulfide_df, no_disulfide_whole_pipeline_df, how="left", left_on=["sequence_name", "model_sequence"], right_on=["sequence_name", "model_sequence"], suffixes=("", "_whole_pipeline"))
    no_disulfide_df.drop(columns=[col for col in no_disulfide_df.columns if "_whole_pipeline" in col], inplace=True)


    variable_linkers_df = pd.read_csv("/home/tadas/code/deltaproteinsBristol/selected_deltaprots/variable_linkers/variable_linkers/variable_linkers_selected_deltaprots.csv")
    variable_linkers_whole_pipeline_df = get_full_pipeline_df("/home/tadas/code/deltaproteinsBristol/pipeline_data/orientation_gap_length_optimisation_optimised")
    variable_linkers_df = pd.merge(variable_linkers_df, variable_linkers_whole_pipeline_df, how="left", left_on=["sequence_name", "model_sequence"], right_on=["sequence_name", "model_sequence"], suffixes=("", "_whole_pipeline"))
    variable_linkers_df.drop(columns=[col for col in variable_linkers_df.columns if "_whole_pipeline" in col], inplace=True)


    no_disulfide_df["name"]="no_disulfide_"+no_disulfide_df["orientation_code"]
    variable_linkers_df["name"]="variable_linkers_"+variable_linkers_df["orientation_code"]

    # merge the two piplene dataframes 
    df = pd.concat([no_disulfide_df, variable_linkers_df], axis=0)

    well_df = pd.read_csv("/home/tadas/code/deltaproteinsBristol/order_optimised_codons_96_wp.csv")

    # merge well_df with df by Name and sort by Well Position
    df = pd.merge(well_df, df, how="left", left_on="Name", right_on="name")



    # MGSSHHHHHHSSGENLYFQSGS addition adds some mw
    df["model_sequence_w_prefix"] = "MGSSHHHHHHSSGENLYFQSGS" + df["model_sequence"]
    df["mass_w_prefix"] = df["model_sequence_w_prefix"].apply(ampal.analyse_protein.sequence_molecular_weight)
    df["sequence_molar_extinction_280_w_prefix"] = df["model_sequence_w_prefix"].apply(ampal.analyse_protein.sequence_molar_extinction_280)
    
    df["percent_helicity_with_prefix"] = df["dssp_assignment"].str.count("H") / df["model_sequence_w_prefix"].str.len()
    
    df.to_pickle("/home/tadas/code/deltaproteinsBristol/deltaprot_designs_data.pkl")

def load_deltaprot_designs_data():
    return pd.read_pickle("/home/tadas/code/deltaproteinsBristol/deltaprot_designs_data.pkl")

def calc_molar_conc(ext_coef, absorbance):
    return ext_coef / absorbance

def calc_mg_per_ml(molar_conc, mass):
    return molar_conc * mass

In [3]:
generate_deltaprot_designs_data()
df = load_deltaprot_designs_data()


/tmp/ipykernel_126087/1357209820.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["model_sequence_w_prefix"] = "MGSSHHHHHHSSGENLYFQSGS" + df["model_sequence"]
/tmp/ipykernel_126087/1357209820.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["mass_w_prefix"] = df["model_sequence_w_prefix"].apply(ampal.analyse_protein.sequence_molecular_weight)
/tmp/ipykernel_126087/1357209820.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor pe

In [ ]:

# populate received_from_idt column. Received everything except for C2,C4,D5,D6,F2,F3,F6


not_received = ['C2', 'C4', 'D5', 'D6', 'F2', 'F3', 'F6']
df['received_from_idt'] = ~df["Well Position"].isin(not_received)

# populate transformation_attempted for (A1-A12,B1-B12,C1,C3,F4,C5,C6)
transformation_attempted_ids = set([f"A{i}" for i in range(1, 13)] +
                    [f"B{i}" for i in range(1, 13)] +
                    ['C1', 'C3', 'F4', 'C5', 'C6'])
df['transformation_attempted'] = df["Well Position"].isin(transformation_attempted_ids)
df['transformation_successful'] = df["Well Position"].isin(transformation_attempted_ids) # all transformations worked, C6 failed initially

# populate expression_levels: (None, "low","medium","high") low: A1,A3,A6,A7,A8,A10,B1. Medium: A5,A11,B2,B3. High: A9,A12


expr_map = {
    # TODO , 'C6' was transformed later, but not yet tested
    **dict.fromkeys(['A2', 'A4', 'B5', 'B6', 'B7', 'B9', 'B11', 'C5'], 'none'), 
    **dict.fromkeys(['A1','A3', 'A6', 'A8', 'B1','B4', 'B10','C3'], 'low'),
    **dict.fromkeys(['A7', 'A10','B12','C1','F4'], 'medium'),
    **dict.fromkeys(['A11', 'B2', 'B3','A5'], 'high'),
    **dict.fromkeys(['A9', 'A12', 'B8'], 'super_high')
}
df['expression_levels'] = df["Well Position"].map(expr_map)

# populate sds_page_size_appearance for every visible band?
# TODO

# populate large_scale_expressed for A12,A9,B8,A1,B12,F4
scaled_expressed_ids = ['A12','A9','B8','A1','B12','F4',"A11","B3","B10"]
df['large_scale_expressed'] = df["Well Position"].isin(scaled_expressed_ids)

# populate scaled_purified for A12,A9,B8 TODO add ,'A1','B12','F4'
# scaled_purified_ids = ['A12','A9','B8'] \

# Note: A9,B8 had some aggregation (small pellet after spining down). A12 did not

A280_after_NI_NTA_IMAC = {
    "A12": 13.41,
    "A9": 6.2,
    "B8": 16.11,
    #"A1" Not Measured, 
    #"B12" Not Measured, 
    #"F4": Not Measured, 
    "A11":6.565,
    "B3":9.067,
    "B10":5.780,
}


df['A280_after_NI_NTA_IMAC'] = df["Well Position"].map(A280_after_NI_NTA_IMAC)
volume_after_NI_NTA_IMAC = 3.2
# calculate molar conc asuming sample is entirely my protein df.sequence_molar_extinction_280

df['molar_conc_after_NI_NTA_IMAC'] = df['A280_after_NI_NTA_IMAC'] / df['sequence_molar_extinction_280_w_prefix']
df['mg_per_ml_after_NI_NTA_IMAC'] = df['molar_conc_after_NI_NTA_IMAC'] * df['mass_w_prefix']
df['mg_after_NI_NTA_IMAC'] = df['mg_per_ml_after_NI_NTA_IMAC'] * volume_after_NI_NTA_IMAC

# after SEC (central fractions)
A280_after_SEC = {
    "A12": 4.93,
    "A9": 2.18,
    "B8": 6.758,
    "A1": 0.820,
    "B12": 2.510,
    "F4": 2.427, 
    "A11":2.478,
    "B3":2.939,
    "B10":2.556,
}
volume_after_sec = {
    "A12": 5.1,
    "A9": 5.1,
    "B8": 3.4,
    "A1": 3.4,
    "B12": 5.1,
    "F4": 5.1,
    "A11": 5.1,
    "B3":3.4,
    "B10":3.4,
}
df["volume_after_sec"] = df["Well Position"].map(volume_after_sec)

df['A280_after_SEC'] = df["Well Position"].map(A280_after_SEC)
df['molar_conc_after_SEC'] = df['A280_after_SEC'] / df['sequence_molar_extinction_280_w_prefix']
df['mg_per_ml_after_SEC'] = df['molar_conc_after_SEC'] * df['mass_w_prefix']
df['mg_after_SEC'] = df['mg_per_ml_after_SEC'] * df['volume_after_sec']


# Biophysical characterisation info (CD and biophysical SEC used same sample. Was assumed it is 20 uM, however it was a bit less.)
cd_sample_A280 = {
    "A12": 0.395, # later measure: 0.406
    "A9": 0.168, # later measure: 0.190
    "B8": 0.517, # later measure: 0.537
    "A1":0.173,
    "B12": 0.210,
    "F4": 0.211,
    "A11": 0.569,
    "B3":0.764,
    "B10":0.175,
}

df["cd_sample_A280"] = df["Well Position"].map(cd_sample_A280)
df["cd_sample_molar_conc"] = df["cd_sample_A280"] / df["sequence_molar_extinction_280_w_prefix"]
df["cd_path_length_mm"]=1

saxs_submitted_samples = ["A1","A9","A12","B8","B12","F4"]

df["saxs_submitted"] = df["Well Position"].isin(saxs_submitted_samples)

# crystal screens set up with 3+3 uL

    # # plate 1, position 2:
    # "B12": 19.19,

    # # plate 2, position 1:
    # "A12": 36.56,
    # # plate 2, position 2:
    # "B8": 43.35,

    # # plate 3, position 1:
    # "A1": 13.82,
    # # plate 3, position 2:
    # "B3": 37.62,
crystal_screens_info = {
    # plate 1, position 1:
    "A9": {
        "plate": 1,
        "position": 1,
        "date": "2025-04-11",
        "A280": 19.40
    },
    "B12":{
        "plate": 1,
        "position": 2,
        "date": "2025-04-11",
        "A280": 19.19
    },
    "A12":{
        "plate": 2,
        "position": 1,
        "date": "2025-04-14",
        "A280": 36.56
    },
    "B8":{
        "plate": 2,
        "position": 2,
        "date": "2025-04-14",
        "A280": 43.35
    },
    "A1":{
        "plate": 3,
        "position": 1,
        "date": "2025-04-17",
        "A280": 13.82
    },
    "B3":{
        "plate": 3,
        "position": 2,
        "date": "2025-04-17",
        "A280": 37.62
    }
}
# unpack all of these columns to df
df["crystal_screens_info"] = df["Well Position"].map(crystal_screens_info)
crystal_keys = ["plate", "position", "date", "A280"]
for key in crystal_keys:
    df[f"crystal_{key}"] = df["crystal_screens_info"].map(lambda x: x.get(key) if isinstance(x, dict) else None)

# Drop original nested column
df.drop(columns=["crystal_screens_info"], inplace=True)
# calculate crystal_mg_per_ml
# df['molar_conc_after_SEC'] = df['A280_after_SEC'] / df['sequence_molar_extinction_280_w_prefix']
# df['mg_per_ml_after_SEC'] = df['molar_conc_after_SEC'] * df['mass_w_prefix']
df["crystal_molar_conc"] = df["crystal_A280"] / df["sequence_molar_extinction_280_w_prefix"]
df["crystal_mg_per_ml"] = df["crystal_molar_conc"] * df["mass_w_prefix"]


# df["mg_per_ml_crystal_screens"] = df["crystal_screens_info"].map(lambda x: x["mg_per_ml"] if x else np.nan)
# df["plate_crystal_screens"] = df["crystal_screens_info"].map(lambda x: x["plate"] if x else np.nan)
# df["position_crystal_screens"] = df["crystal_screens_info"].map(lambda x: x["position"] if x else np.nan)
# df["date_crystal_screens"] = df["crystal_screens_info"].map(lambda x: x["date"] if x else np.nan)
    
                 
experimental_cols = [
    "model_sequence_w_prefix",
    "mass_w_prefix",
    "sequence_molar_extinction_280_w_prefix",
    "received_from_idt",
    "transformation_attempted",
    "transformation_successful",
    "expression_levels",
    "large_scale_expressed",
    "A280_after_NI_NTA_IMAC",
    "molar_conc_after_NI_NTA_IMAC",
    "mg_per_ml_after_NI_NTA_IMAC",
    "mg_after_NI_NTA_IMAC",
    "volume_after_sec",
    "A280_after_SEC",
    "molar_conc_after_SEC",
    "mg_per_ml_after_SEC",
    "mg_after_SEC",
    "cd_sample_A280",
    "cd_sample_molar_conc",
    "cd_path_length_mm",
    "saxs_submitted",
    "crystal_plate", "crystal_position", "crystal_date", "crystal_A280","crystal_molar_conc","crystal_mg_per_ml"
    # "scaled_purified" is mentioned but not actually created
]


/tmp/ipykernel_126087/3285908694.py:166: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["crystal_screens_info"] = df["Well Position"].map(crystal_screens_info)


In [7]:
df[useful_cols+experimental_cols].to_csv("/home/tadas/code/deltaproteinsBristol/experimental_results/deltaprot_designs_data_with_results.csv", index=False)
df[sharing_cols+experimental_cols].to_csv("/home/tadas/code/deltaproteinsBristol/experimental_results/deltaprot_designs_data_with_results_minimised.csv", index=False)
df.to_pickle("/home/tadas/code/deltaproteinsBristol/deltaprot_designs_all_data_with_results.pkl")

In [10]:
df[df["large_scale_expressed"]][["Well Position","crystal_plate"]]

,Well Position,crystal_plate
0,A1,3.0
8,A9,1.0
10,A11,NaN
11,A12,2.0
14,B3,3.0
19,B8,2.0
21,B10,NaN
23,B12,1.0
57,F4,NaN


In [ ]:
df[df['large_scale_expressed']][["Well Position","Name","mass_w_prefix","expression_levels","mg_per_ml_after_SEC","mg_after_SEC","molar_conc_after_SEC","cd_sample_molar_conc"]]

In [3]:
df

NameError: name 'df' is not defined

In [ ]:
# set limit to column width
pd.set_option('display.max_colwidth', 1)

In [ ]:
df[["Well Position","cd_sample_molar_conc","Name","expression_levels"]]

In [ ]:
df[["Well Position","cd_sample_molar_conc","Name","A280_after_SEC","volume_after_sec","sequence_molar_extinction_280_w_prefix","molar_conc_after_SEC","mg_per_ml_after_SEC","mg_after_SEC"]]

In [ ]:
df[["Well Position","Name","mass","mass_w_prefix","charge","A280_after_SEC","volume_after_sec","sequence_molar_extinction_280_w_prefix","molar_conc_after_NI_NTA_IMAC","molar_conc_after_SEC","mg_per_ml_after_SEC","mg_after_SEC"]]

In [ ]:
# mass correct: A9, A11, B2, C1, F4, B12
# mass questionable: A12 (has 2 bands), B3 (2 bands)
# mass wrong: 

In [ ]:
# unlimit pandas column display number
pd.set_option('display.max_columns', None)

In [15]:
df[df["transformation_successful"] & df["expression_levels"].isin(["low","medium","high","super_high", np.nan]) & ~df["large_scale_expressed"]][["Well Position","Name","expression_levels"]]

,Well Position,Name,expression_levels
2,A3,no_disulfide_b4iiiix,low
4,A5,no_disulfide_b4iiin,high
5,A6,no_disulfide_b4inin,low
6,A7,no_disulfide_b4innn,medium
7,A8,no_disulfide_b4nnnnx,low
9,A10,no_disulfide_b5iiiin,medium
12,B1,no_disulfide_b5innnn,low
13,B2,no_disulfide_b6iiniin,high
15,B4,no_disulfide_b6inninn,low
24,C1,no_disulfide_l5inni,medium


In [12]:
df[df["expression_levels"].isin(["low","medium","high","super_high"])].sort_values(["large_scale_expressed","expression_levels"],ascending=[False,True])[["Well Position","Name","mass_w_prefix","expression_levels","large_scale_expressed"]]

,Well Position,Name,mass_w_prefix,expression_levels,large_scale_expressed
10,A11,no_disulfide_b5iinin,14240.78108,high,True
14,B3,no_disulfide_b6ininin,15691.18198,high,True
0,A1,no_disulfide_b3iii,9661.41738,low,True
21,B10,no_disulfide_l4iin,10960.05638,low,True
23,B12,no_disulfide_l5iiin,12669.91628,medium,True
57,F4,variable_linkers_l6innni,11020.77008,medium,True
8,A9,no_disulfide_b4nnnny,11094.27778,super_high,True
11,A12,no_disulfide_b5ininn,14092.83838,super_high,True
19,B8,no_disulfide_h6i_i_i,16168.21468,super_high,True
4,A5,no_disulfide_b4iiin,9348.14028,high,False


In [23]:
df[df["expression_levels"].isin(["low","medium","high","super_high"]) & ~df["large_scale_expressed"]][ ["Well Position","expression_levels"]]

,Well Position,expression_levels
2,A3,low
4,A5,high
5,A6,low
6,A7,medium
7,A8,low
9,A10,medium
12,B1,low
13,B2,high
15,B4,low
24,C1,medium


In [17]:
df[df["expression_levels"].isin(["low","medium","high","super_high"])]

,Well Position,Name,Sequence,orientation_code,atp_cost_per_aa,dna_complexity_per_aa,pll,pll_per_aa,isoelectric_point,charge,sequence_charge,mass,sequence_length,mean_plddt,mean_pae,ptm,tm_rmsd,tm_score_assembly,tm_score_design,tm_rmsd100,dp_finder_total_cost_norm,predicted_usability,combined_score,path_score_version,residues_per_helix,deltahedron_edge_length,diffuse_termini,avoid_amino_acids,increase_amino_acid_likelihood,energy_minimization,algorithms_sequence_prediction,algorithms_structure_prediction,rf_file_num,rib_num,aa_count_per_gap,model_sequence,sequence_molar_extinction_280,sequence_molecular_weight,backbone_loop_mask_string,aligned_length,seq_id,sequence_name,dssp_assignment,aggrescan3d_avg_value,hydrophobic_fitness,packing_density,rosetta_total_per_aa,name,structure_prediction_file_name,structure_prediction_file_path,deltaprot_score,loop_lengths,designs_per_orientation,rf_diffusion_mask_input_seq_bool,directories_assembly_inputs,directories_rf_diffusion_outputs,directories_seq_pred_inputs,directories_seq_pred_outputs,directories_structure_prediction,directories_evaluation,composition_ALA,composition_CYS,composition_ASP,composition_GLU,composition_PHE,composition_GLY,composition_HIS,composition_ILE,composition_LYS,composition_LEU,composition_MET,composition_ASN,composition_PRO,composition_GLN,composition_ARG,composition_SER,composition_THR,composition_VAL,composition_TRP,composition_TYR,atp_cost,dna_complexity,aa_count_per_gap_list,index_helix1_n,index_helix1_c,index_helix2_n,index_helix2_c,index_helix3_n,index_helix3_c,index_helix4_n,index_helix4_c,index_helix5_n,index_helix5_c,index_helix6_n,index_helix6_c,linker1_length,linker2_length,linker3_length,linker4_length,linker5_length,deltahedron_name,taylor_letter_packing_descriptors,chothia_omega_angles,overall_distance_score_v2,overall_path_distance_score_v2,overall_linker_length_score,overall_contact_order_score,overall_linker_convenience_score,overall_linker_convenience_v2_score,path_score_v2,path_score_v3,angle_helix1,hydrophobic_count_helix1,residue_count_helix1,angle_helix2,hydrophobic_count_helix2,residue_count_helix2,angle_helix3,hydrophobic_count_helix3,residue_count_helix3,angle_helix4,hydrophobic_count_helix4,residue_count_helix4,angle_helix5,hydrophobic_count_helix5,residue_count_helix5,angle_helix6,hydrophobic_count_helix6,residue_count_helix6,angle_resolution,path,design_name,file_name,full_sequence,composition_UNK,ss_prop_alpha_helix,ss_prop_beta_bridge,ss_prop_beta_strand,ss_prop_3_10_helix,ss_prop_pi_helix,ss_prop_hbonded_turn,ss_prop_bend,ss_prop_loop,num_residues,budeff_total,budeff_steric,budeff_desolvation,budeff_charge,evoef2_total,evoef2_ref_total,evoef2_intraR_total,evoef2_interS_total,evoef2_interD_total,dfire2_total,rosetta_total,rosetta_fa_atr,rosetta_fa_rep,rosetta_fa_intra_rep,rosetta_fa_elec,rosetta_fa_sol,rosetta_lk_ball_wtd,rosetta_fa_intra_sol_xover4,rosetta_hbond_lr_bb,rosetta_hbond_sr_bb,rosetta_hbond_bb_sc,rosetta_hbond_sc,rosetta_dslf_fa13,rosetta_rama_prepro,rosetta_p_aa_pp,rosetta_fa_dun,rosetta_omega,rosetta_pro_close,rosetta_yhh_planarity,aggrescan3d_total_value,aggrescan3d_min_value,aggrescan3d_max_value,hydrophobic_fitness_per_aa,budeff_total_per_aa,budeff_steric_per_aa,budeff_desolvation_per_aa,budeff_charge_per_aa,evoef2_total_per_aa,evoef2_ref_total_per_aa,evoef2_intraR_total_per_aa,evoef2_interS_total_per_aa,evoef2_interD_total_per_aa,dfire2_total_per_aa,rosetta_fa_atr_per_aa,rosetta_fa_rep_per_aa,rosetta_fa_intra_rep_per_aa,rosetta_fa_elec_per_aa,rosetta_fa_sol_per_aa,rosetta_lk_ball_wtd_per_aa,rosetta_fa_intra_sol_xover4_per_aa,rosetta_hbond_lr_bb_per_aa,rosetta_hbond_sr_bb_per_aa,rosetta_hbond_bb_sc_per_aa,rosetta_hbond_sc_per_aa,rosetta_dslf_fa13_per_aa,rosetta_rama_prepro_per_aa,rosetta_p_aa_pp_per_aa,rosetta_fa_dun_per_aa,rosetta_omega_per_aa,rosetta_pro_close_per_aa,rosetta_yhh_planarity_per_aa,dp_finder_filename,dp_finder_helix_axes,dp_finder_orientation_code,dp_finder_rotation_matrix,dp_finder_scale,dp_finder_tr

In [23]:
df[df["expression_levels"].isin(["low"])]["Well Position"]


0      A1
2      A3
5      A6
7      A8
12     B1
15     B4
21    B10
26     C3
Name: Well Position, dtype: object

In [ ]:
# Choices for large scale expression

# A12	no_disulfide_b5ininn super_high
# A9	no_disulfide_b4nnnny
# B8	no_disulfide_h6i_i_i

# A1 no_disulfide_b3iii (low-medium)
# B12	no_disulfide_l5iiin (medium)
# F4	variable_linkers_l6innni	(medium)


In [ ]:
p